# Fetch data

In [ ]:
import pld
from pathlib import Path
import ipywidgets as ipw
from IPython.display import clear_output
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

In [ ]:
DATADIR = Path("../") / "data"
DATASET = "20250127_170419"
DATAPATH = (DATADIR / DATASET).absolute()

In [ ]:
analyzer = pld.RHEEDAnalyzer()
analyzer.set_outdir(DATAPATH / "output")

In [ ]:
analyzer.load_data(DATAPATH)

In [ ]:
analyzer.crop_to_global_region_of_interest()

In [ ]:
frame = pld.RHEEDFrame(0, analyzer.data[0])

In [ ]:
analyzer.generate_frames(
    sigma=1,
    params={
        "min_distance": 10,
        "threshold_rel": 0.1,
    }
)

# Extract time series

In [ ]:
# intensities = analyzer.get_peak_intensities(sigma=1)

In [ ]:
# import csv

# with open("central_peak_intensity_time_series.csv", "w") as f:
#     writer = csv.writer(f)
#     writer.writerow(["time", "intensity"])
#     for t, i in zip(analyzer.timestamps, intensities[0]):
#         writer.writerow([t, i])

## RHEED frame analysis

### Single frame check

In [ ]:
def plot_frame(frame: pld.RHEEDFrame):
    clear_output(wait=True)
    frame.plot(
        show_peaks=True,
        show_regions_of_interest=False,
        figsize=(4, 3),
    )


plot_frame(analyzer.frames[0])

rheed_slider = ipw.IntSlider(min=0, max=len(analyzer.frames) - 1)


def on_slider_change(change):
    plot_frame(analyzer.frames[change["new"]])


rheed_slider.observe(on_slider_change, names="value")

In [ ]:
rheed_slider

In [ ]:
def plot_spectrum(frame: pld.RHEEDFrame):
    spectrum = frame.get_center_peak_cross_section()
    peaks, _ = find_peaks(spectrum, distance=20 // 2, prominence=25)

    clear_output(wait=True)
    plt.figure(figsize=(8, 4))
    plt.plot(spectrum, label="Vertical Intensity Profile")
    plt.scatter(peaks, spectrum[peaks], color="red", label="Detected Peaks")
    # plt.ylim(2, 10)
    plt.show()


plot_spectrum(analyzer.frames[0])

cross_section_slider = ipw.IntSlider(min=0, max=len(analyzer.frames) - 1)


def on_slider_change(change):
    plot_spectrum(analyzer.frames[change["new"]])


cross_section_slider.observe(on_slider_change, names="value")

In [ ]:
cross_section_slider

In [ ]:
import plotly.graph_objects as go

ratios = [frame.get_peak_ratio_sum(prominence=25) for frame in analyzer.frames]

fig = go.Figure()
fig.add_trace(go.Scatter(y=ratios, mode="markers"))
fig.update_layout(
    xaxis=dict(range=[0, len(ratios)]),
)

In [ ]:
# frame.plot_power_spectrum(figsize=(4, 3))

# Time series

In [ ]:
analyzer.plot_sharpness_time_series(
    data=analyzer.get_sharpness(),
    figsize=(8, 4),
)

In [ ]:
analyzer.plot_radial_profile_time_series(
    data=analyzer.get_radial_profile(),
    figsize=(8, 4),
)

In [ ]:
import numpy as np

analyzer.plot_intensity_time_series(
    data=np.array(analyzer.get_peak_intensities()) / 255,
    figsize=(4, 4),
)

# Score

In [ ]:
analyzer.compute_deposition_score()

In [ ]:
# analyzer.analyze_quality(0)

In [ ]:
assert False

# Optimization

In [ ]:
from bayes_opt import BayesianOptimization

from bayes_opt import acquisition

acq = acquisition.UpperConfidenceBound(kappa=2.5)


def evaluate_growth_quality(
    gas_pressure,
    lase_fluence,
    substrate_temperature,
    i=0,
):
    """
    Replace this with your actual function that evaluates growth quality.
    It could run an experiment or use a surrogate model.
    """
    # Placeholder: Assume we have a function that scores growth quality
    score = analyzer.analyze_quality(i)
    return score


pbounds = {
    "gas_pressure": (1e-6, 1e-2),  # Torr
    "lase_fluence": (0.1, 1.0),  # J/cm^2
    "substrate_temperature": (500, 800),  # K
}

optimizer = BayesianOptimization(
    f=None,
    acquisition_function=acq,
    pbounds=pbounds,
    verbose=0,
    random_state=42,
)

max_iter = 10
for i in range(10):
    next_point = optimizer.suggest()
    result = evaluate_growth_quality(**next_point, i=i % 3)
    score = result["decay_rate"]
    optimizer.register(params=next_point, target=score)
    print(score, next_point)

print(optimizer.max)

# egobox